In [6]:
import numpy as np
from tqdm import tqdm
import cv2
import os
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score

from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from keras.layers import Dense, Dropout, Flatten, Input, BatchNormalization, Conv2D, MaxPooling2D
from keras.models import Model
from keras import metrics

from sklearn import metrics as sklearn_metrics

from google.colab.patches import cv2_imshow

In [ ]:
!mkdir /content/images
!mkdir /content/images/train
!mkdir /content/labels
!mkdir /content/labels/train

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [4]:
!unzip -q '/content/drive/My Drive/NATO/train_base.zip' -d '/content/images/train'
!unzip -q '/content/drive/My Drive/NATO/train_base_l.zip' -d '/content/labels/train'

{'otse_k': 4362, 'otse_l': 4349, 'parem_n': 5141, 'parem_y': 5103, 'vasak_n': 5173, 'vasak_y': 5162}

In [20]:
def CNNModel():
    classes = 6

    input = Input(shape=(80, 80, 1))
    x = Conv2D(64, kernel_size = (3, 3), strides = (1, 1), padding = 'same', activation='relu')(input)
    x = MaxPooling2D(pool_size=(2, 2), padding = 'same')(x)
    x = BatchNormalization()(x)
    x = Conv2D(64, kernel_size = (3, 3), strides = (1, 1), padding = 'same', activation='relu')(x)
    x = MaxPooling2D(pool_size=(2, 2), padding = 'same')(x)
    x = BatchNormalization()(x)
    x = Conv2D(128, kernel_size = (3, 3), strides = (1, 1), padding = 'same', activation='relu')(x)
    x = MaxPooling2D(pool_size=(2, 2), padding = 'same')(x)
    x = BatchNormalization()(x)
    x = Conv2D(128, kernel_size = (3, 3), strides = (1, 1), padding = 'same', activation='relu')(x)
    x = MaxPooling2D(pool_size=(2, 2), padding = 'same')(x)
    x = BatchNormalization()(x)
    x = Dense(256, activation='relu')(x)
    x = Dense(256, activation='relu')(x)
    x = Flatten()(x)
    x = Dense(classes, activation='softmax')(x)

    model = Model(inputs=input, outputs=x)
    print(model.summary())

    return model

In [ ]:
images = '/content/images/train'
labels = '/content/labels/train'

x_ = []
y_ = []

file_list = os.listdir(images)

for name in tqdm(file_list):
    image = cv2.imread(images+"/"+name, cv2.IMREAD_GRAYSCALE)
    with open(labels+"/"+str(name).split(".")[0]+".txt") as f:
        lines = [line.rstrip('\n') for line in f]
        for line in lines:
            l = int(line.split(" ")[0])
            x = int((float(line.split(" ")[1])-(float(line.split(" ")[3])/2)) * 770)
            y = int((float(line.split(" ")[2])-(float(line.split(" ")[4])/2)) * 578)
            w = int(float(line.split(" ")[3]) * 770 + 5)
            h = int(float(line.split(" ")[4]) * 576 + 5)

            crop_image = image[y:y+h, x:x+w]
            dim = (80, 80)
            resized_img = cv2.resize(crop_image, dim, interpolation = cv2.INTER_AREA)
            x_.append(resized_img)
            y_.append(l)

In [ ]:
y_categorical = to_categorical(y_, 6)

x_train = np.array(x_[0:])
y_train = np.array(y_categorical)

x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], 1)

print(x_train.shape, y_train.shape)

model = CNNModel()

model.compile(loss='categorical_crossentropy',
                optimizer='adam',
                metrics=[metrics.categorical_accuracy])

model.fit(x_train, y_train, batch_size = 64, epochs = 50)

In [22]:
model.save("base_pose_model_1803_5.h5")

In [ ]:
!mkdir /content/images
!mkdir /content/images/test
!mkdir /content/labels
!mkdir /content/labels/test

In [13]:
!unzip -q '/content/drive/My Drive/NATO/test_base.zip' -d '/content/images/test'
!unzip -q '/content/drive/My Drive/NATO/test_base_l.zip' -d '/content/labels/test'

{'otse_k': 997, 'otse_l': 987, 'parem_n': 1013, 'parem_y': 1024, 'vasak_n': 1051, 'vasak_y': 1009}

In [ ]:
images_test = '/content/images/test'
labels_test = '/content/labels/test'

file_list = os.listdir(images_test)
model = CNNModel()

actual = []
predicted = []

for name in tqdm(file_list):
    image = cv2.imread(images_test+"/"+name, cv2.IMREAD_GRAYSCALE)
    with open(labels_test+"/"+str(name).split(".")[0]+".txt") as f:
        lines = [line.rstrip('\n') for line in f]
        for line in lines:
            actual.append(int(line.split(" ")[0]))

            x = int((float(line.split(" ")[1])-(float(line.split(" ")[3])/2)) * 770)
            y = int((float(line.split(" ")[2])-(float(line.split(" ")[4])/2)) * 578)
            w = int(float(line.split(" ")[3]) * 770 + 5)
            h = int(float(line.split(" ")[4]) * 576 + 5)

            # Symbol
            crop_image = image[y:y+h, x:x+w]
            dim = (80, 80)
            resized_img = cv2.resize(crop_image, dim, interpolation = cv2.INTER_AREA)

            ## Predict
            reshaped_img = resized_img.reshape(1, 80, 80)
            model.load_weights('/content/base_pose_model_1803.h5')
            predicted_label = model.predict(reshaped_img, verbose=0)
            predicted.append(np.argmax(predicted_label))

In [ ]:
confusion_matrix = sklearn_metrics.confusion_matrix(actual, predicted)
cm_display = sklearn_metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = ['otse_kitsas', 'otse_lai', 'parem_45', 'parem_90', 'vasak_45', 'vasak_90'])

fig, ax = plt.subplots(figsize=(10,10))
cm_display.plot(ax = ax)

print("Accuracy:", accuracy_score(actual, predicted))